# 3 УРОК

ОБЪЕДИНЕНИЕ ТАБЛИЦ – JOIN

## ШАГ 4

У пользователя может быть два идентификатора – UserID и DeviceID. В таблице checks есть только UserID, в остальных – только DeviceID. Во вспомогательной таблице devices есть и UserID, и DeviceID. 

Давайте с помощью JOIN дополним таблицу events (left) данными о UserID пользователей из таблицы devices (right). Для некоторых DeviceID не будет пары UserID из таблицы devices – подумайте, какой вид JOIN подойдет, чтобы не потерять те строки, где DeviceID есть в events, но нет в devices.

Укажите UserID из первой строки результирующей таблицы, используя сортировку по убыванию по полю DeviceID.

Подсказки

In [ ]:
SELECT *
FROM events as l
LEFT JOIN 
    default.devices as r
    ON l.DeviceID = r.DeviceID

ORDER BY l.DeviceID DESC
LIMIT 12

##### ОТВЕТ : 8158889386484182099

## ШАГ 5
### Давайте проверим, пользователи пришедшие из какого источника совершили наибольшее число покупок. В качестве ответа выберите название Source, юзеры которого совершили больше всего покупок.

Hint: Для этого используйте UserID, DeviceID и Source из соответствующих таблиц. Считать уникальные значения здесь не нужно. Также заметьте, что покупки со стоимостью 0 рублей всё ещё считаются покупками.

SELECT Source,
    COUNT(Source) AS Count_Sourse
FROM 
(
SELECT 
    l.DeviceID AS DeviceID,
    c.Source AS Source,
    r.UserID AS UserID
FROM events AS l
LEFT JOIN devices AS r
    ON l.DeviceID = r.DeviceID
JOIN installs as c
    ON l.DeviceID = c.DeviceID
ORDER BY DeviceID DESC
--LIMIT 100
)
GROUP BY Source
ORDER BY Count_Sourse DESC
LIMIT 100

#### ИЗ РАЗБОРА

In [ ]:
SELECT 
    
    l.Source AS Source_,
    COUNT(*) AS SELL_Count
FROM installs AS l
    JOIN devices AS r
        ON l.DeviceID = r.DeviceID
    JOIN checks as c
    ON c.UserID = r.UserID
GROUP BY Source_
ORDER BY SELL_Count DESC


##### Ответы сошлись но  количества совершенно разные

#### Ответ Source_27

## ШАГ 6
### Теперь выясним, сколько всего уникальных юзеров совершили покупки в нашем приложении.

Объедините нужные таблицы, посчитайте число уникальных UserID для каждого источника (Source), и в качестве ответа укажите число пользователей, пришедших из Source_7.

Hint: checks – покупки, devices – соответствие, installs – информация об источнике.

Подсказки

ВАЖНОЕ ЗАМЕЧАНИЕ!

Если вы делаете задание через REDASH, убедитесь, что в ответ вносите значение без запятых между порядками числа ( 9,424,234.05 -> 9424234.05 )

In [ ]:
SELECT Source,
    COUNT(DISTINCT UserID)
FROM
(
SELECT 
    r.DeviceID AS DeviceID,
    c.Source AS Source,
    r.UserID AS UserID
FROM checks AS l
    JOIN devices AS r
    ON l.UserID = r.UserID
JOIN installs as c
    ON r.DeviceID = c.DeviceID
ORDER BY DeviceID DESC
--LIMIT 100
)
GROUP BY Source
LIMIT 100

#### из разбора

In [ ]:
SELECT 
    
    l.Source AS Source_,
    uniqExact(c.UserID)
FROM installs AS l
    JOIN devices AS r
        ON l.DeviceID = r.DeviceID
    JOIN checks as c
    ON c.UserID = r.UserID
GROUP BY Source_

#### ответы одинаковые

##### ОТВЕТ 11432

## ШАГ 7
Самое время посмотреть на общую выручку, а также минимальный, максимальный и средний чек. Рассчитайте нужные показатели и соотнесите полученные значения:



In [ ]:
SELECT Source,
    SUM(Rub) AS revenue,
    MAX(Rub) AS MAX_check,
    MIN(Rub) AS MIN_check,
    AVG(Rub) AS AVG_check
FROM
(
SELECT 
    l.Rub AS Rub,
    r.DeviceID AS DeviceID,
    c.Source AS Source,
    r.UserID AS UserID
FROM checks AS l
    JOIN devices AS r
    ON l.UserID = r.UserID
JOIN installs as c
    ON r.DeviceID = c.DeviceID
ORDER BY DeviceID DESC
--LIMIT 100
)
GROUP BY Source
LIMIT 100
--HAVING Source = 'Source_20'

#### ИЗ разбора

In [ ]:
SELECT 
    
    l.Source AS Source_,
    SUM(Rub) AS revenue,
    MAX(Rub) AS MAX_check,
    MIN(Rub) AS MIN_check,
    AVG(Rub) AS AVG_check
FROM installs AS l
    JOIN devices AS r
        ON l.DeviceID = r.DeviceID
    JOIN checks as c
    ON c.UserID = r.UserID
GROUP BY Source_

#### выводы одинаковые...

##### ОТВЕТ
Максимальный чек пользователей из Source_14 38189

Общая сумма денег, полученная от пользователей Source_1 74

Общая выручка для Source_13 2752

Максимальная сумма заказа среди пользователей из Source_10 10

Средний чек пользователей Source_21 5.5555556


## ШАГ 8
#### Выведите идентификаторы устройств пользователей, которые совершили как минимум одну покупку за последний месяц (октябрь 2019). Используйте сортировку по возрастанию DeviceID и укажите минимальное значение.

Hint: для извлечения месяца из даты можно использовать toMonth() или  toStartOfMonth(), предварительно приведя BuyDate к типу date.

In [ ]:
SELECT 
    toStartOfMonth(CAST(l.BuyDate AS DateTime)) AS BuyMonth,
    l.Rub AS Rub,
    r.DeviceID AS DeviceID
    --c.Source AS Source
    --l.UserID AS UserID
FROM checks AS l
    JOIN devices AS r
    ON l.UserID = r.UserID
WHERE Rub > 0 AND BuyMonth = '2019-10-01'

ORDER BY  DeviceID 
LIMIT 100

#### из разбора

In [ ]:
SELECT *
   
FROM checks AS l
    JOIN devices AS r
    ON l.UserID = r.UserID
WHERE toStartOfMonth(CAST(l.BuyDate AS DateTime)) = '2019-10-01'

ORDER BY  DeviceID 
LIMIT 10

#### ОТВЕТ 22374478679519

## ШАГ 9
Проверим, сколько товаров (events) в среднем просматривают пользователи с разных платформ (Platform), и пришедшие из разных источников  (Source). Для этого объедините таблицы events и installs, и посчитайте, сколько просмотров в среднем приходится на каждую пару платформа-канал привлечения.

Отсортируйте полученную табличку по убыванию среднего числа просмотров. В качестве ответа укажите платформу и источник, пользователи которого в среднем просматривали товары бóльшее число раз.

In [ ]:
SELECT 
    r.Platform AS Platform,
    r.Source AS Source,
    AVG(events) AS AVG_events
FROM events AS l
    JOIN installs AS r
    ON l.DeviceID = r.DeviceID
GROUP BY Platform, Source
ORDER BY  AVG_events DESC 
LIMIT 100

#### ОТВЕТ
Платформа IOS номер источника 20

## ШАГ 10
#### Давайте посчитаем число уникальных DeviceID в инсталлах, для которых присутствуют просмотры в таблице events с разбивкой по платформам (поле Platform). Для этого можно отобрать все строки только из таблицы installs, для которых нашлось соответствие в таблице events. 

В качестве ответа введите число инсталлов, для которых нашлись просмотры, на платформе android.

Внимание! "Нашлось 0 просмотров" тоже считается как "нашлись просмотры". Главное, чтобы не было именно пропуска.

In [ ]:
SELECT 
    r.Platform AS Platform,
    COUNT(DISTINCT(r.DeviceID)) AS COUNT_DeviceID   
FROM events AS l
    JOIN installs AS r
    ON l.DeviceID = r.DeviceID
GROUP BY Platform
LIMIT 100

### ИЗ разбора

In [ ]:
SELECT 
    r.Platform AS Platform,
    COUNT(DISTINCT(r.DeviceID)) AS COUNT_DeviceID   
FROM installs AS r
LEFT SEMI JOIN events AS l 
    ON l.DeviceID = r.DeviceID
GROUP BY Platform
LIMIT 100

#### Одинаково

#### Разьяснения от Support с курсов

Добрый день! Не совсем так. Если у нас такая ситуация, что значениям колонки в левой таблице соответствует несколько значений из правой, то LEFT SEMI JOIN вернет только одну строку из левой таблицы, в то время как INNER JOIN вернет столько строк, сколько найдет соответствий в правой таблице. 
Здесь у вас нет разницы, так как выполнена группировка и в любом случае вы считаете все уникальные девайс айди на каждую платформу, поэтому в данной задаче действительно не так принципиально, какой джойн использовать.

#### Ответ

9051220

## ШАГ 11
#### Давайте теперь посчитаем конверсию из инсталла в просмотр с разбивкой по платформе инсталла – в данном случае это доля DeviceID, для которых есть просмотры, от всех DeviceID в инсталлах. 

Для этого нужно объединить таблицы installs и events так, чтобы получить все DeviceID инсталлов и соответствующие им DeviceID из events, посчитать число уникальных DeviceID инсталлов (1) и соответствующих DeviceID из events (2) и вычислить долю (2) от (1). В качестве ответа укажите значение конверсии из инсталла в просмотр на платформе ios. 

Внимание: ответ указать не в процентах, а именно в виде доли (т.е. не нужно домножать полученный ответ на 100). 

In [ ]:
SELECT 
    r.Platform AS Platform,
    COUNT(DISTINCT(r.DeviceID)) AS COUNT_DeviceID_INST,
    COUNT(DISTINCT(l.DeviceID)) AS COUNT_DeviceID_EVEN,
    COUNT_DeviceID_EVEN/COUNT_DeviceID_INST AS CTR
FROM events AS l
    RIGHT JOIN installs AS r
    ON l.DeviceID = r.DeviceID
GROUP BY Platform
LIMIT 100

#### из разбора (ПОЧЕМУ то не разбил по платформам)

In [ ]:
SELECT 
    uniqExact(l.DeviceID)/uniqExact(r.DeviceID )
    
FROM events AS l
    RIGHT JOIN installs AS r
    ON l.DeviceID = r.DeviceID
--GROUP BY Platform
--LIMIT 100

0.983182191473886

In [2]:
2031629/2066381

0.983182191473886

#### Ответ : 0.983182191473886


## ШАГ 12
### Представим, что в логирование DeviceID в событиях закралась ошибка - часть ID была записана в базу некорректно. Это привело к тому, что в таблице с событиями появились DeviceID, для которых нет инсталлов. Нам надо отобрать примеры DeviceID из таблицы event, которых нет в таблице installs, чтобы отправить их команде разработчиков на исправление. 

Выведите 10 уникальных DeviceID, которые присутствуют в таблице events, но отсутствуют в installs, отсортировав их в порядке убывания. В качестве ответа введите первый DeviceID из списка.  

In [ ]:
SELECT l.DeviceID AS DeviceID
    
FROM events AS l
    LEFT ANTI JOIN installs AS r
    ON l.DeviceID = r.DeviceID

ORDER BY DeviceID DESC
LIMIT 10

###### ОТВЕТ
18446741643567719656

In [ ]:
Там это ID много раз

In [2]:
2**64

18446744073709551616